In [1]:
import numpy as np
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
import string

# Imports
import pandas as pd

import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.es import Spanish

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

import pickle

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

nltk.download('stopwords')
nltk.download('punkt')

# https://www.aurigait.com/blog/recommendation-system-using-knn/
# https://www.analyticsvidhya.com/blog/2020/08/recommendation-system-k-nearest-neighbors/
# https://www.mygreatlearning.com/blog/nltk-tutorial-with-python/
# https://www.datacamp.com/community/tutorials/recommender-systems-python
# https://stackabuse.com/python-for-nlp-tokenization-stemming-and-lemmatization-with-spacy-library/
# https://es.stackoverflow.com/questions/331602/c%C3%B3mo-lematizar-en-espa%C3%B1ol-una-string-con-nltk
# https://medium.com/@jae.finger/building-a-marijuana-strain-recommender-system-bef3f66bb2d3
# https://scikit-learn.org/stable/modules/neighbors.html

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
file = open("Documentos.txt", "r", encoding='utf8')
lines = file.readlines()

datos = {'Género':['Salud', 'Salud', 'Salud', 'Cine', 'Ciencia', 'Salud', 'Salud'],
        'Ratings':[7,5,6,5,6,8,6],
        'Documentos':lines}
df = pd.DataFrame(data = datos)
df.head(10)

,Género,Ratings,Documentos
0,Salud,7,Los grupos musculares se ejercitan en función ...
1,Salud,5,Existen dos tipos de actividades física que se...
2,Salud,6,Los deportes más practicados a nivel mundial s...
3,Cine,5,Las películas con más Oscars de la historia so...
4,Ciencia,6,El sol es una estrella que irradia calor y ene...
5,Salud,8,Practicar deporte es altamente recomendado par...
6,Salud,6,Se han realizado varios estudios reflejando la...


In [3]:
df_copy = df.copy()
for i,x in enumerate(df_copy['Documentos']):
        df_copy['Documentos'][i] = df_copy['Documentos'][i][:-2]
        print(df_copy['Documentos'][i])

df['Documentos'] = df_copy['Documentos']

Los grupos musculares se ejercitan en función del ejercicio específico que se realice, por ejemplo, si quieres realizar un ejercicio de alta intensidad, optarás por uno anaeróbico y viceversa
Existen dos tipos de actividades física que se suelen practicar, los ejercicios aeróbicos y los ejercicios anaeróbicos
Los deportes más practicados a nivel mundial son el fútbol y el baloncesto, donde se fomenta el compañerismo y el respeto
Las películas con más Oscars de la historia son Titanic, BenHur y El Señor de los Anillos
El sol es una estrella que irradia calor y energía por toda la vía lactea. Es parte fundamental del sistema solar
Practicar deporte es altamente recomendado para mejorar la salud física y mental, según han publicado varias revistas y organismos
Se han realizado varios estudios reflejando la importancia del ejericio como medida para contrarrestar el estrés


C:\Users\Usuario\Anaconda2\envs\env_python_3_6\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
df['Genero_+_documento'] = df['Género']+ ' ' + df['Documentos']

df['Genero_+_documento'] = df['Genero_+_documento'].apply(lambda x: str.lower(x))

In [5]:
# https://www.mygreatlearning.com/blog/nltk-tutorial-with-python/
# https://datagy.io/python-remove-punctuation-from-string/
    
### Primero tokenizamos los 4 primeros documentos
def tokenize_words(text):
    sin_puntuacion = text.translate(str.maketrans('', '', string.punctuation))
    tokenized_text=word_tokenize(sin_puntuacion)
    print(tokenized_text)
    print(type(tokenized_text))
    
    return tokenized_text

# df_tokenized = df.copy()
df['Words_tokenized'] = df['Genero_+_documento'].apply(lambda x: tokenize_words(x))
    


['salud', 'los', 'grupos', 'musculares', 'se', 'ejercitan', 'en', 'función', 'del', 'ejercicio', 'específico', 'que', 'se', 'realice', 'por', 'ejemplo', 'si', 'quieres', 'realizar', 'un', 'ejercicio', 'de', 'alta', 'intensidad', 'optarás', 'por', 'uno', 'anaeróbico', 'y', 'viceversa']
<class 'list'>
['salud', 'existen', 'dos', 'tipos', 'de', 'actividades', 'física', 'que', 'se', 'suelen', 'practicar', 'los', 'ejercicios', 'aeróbicos', 'y', 'los', 'ejercicios', 'anaeróbicos']
<class 'list'>
['salud', 'los', 'deportes', 'más', 'practicados', 'a', 'nivel', 'mundial', 'son', 'el', 'fútbol', 'y', 'el', 'baloncesto', 'donde', 'se', 'fomenta', 'el', 'compañerismo', 'y', 'el', 'respeto']
<class 'list'>
['cine', 'las', 'películas', 'con', 'más', 'oscars', 'de', 'la', 'historia', 'son', 'titanic', 'benhur', 'y', 'el', 'señor', 'de', 'los', 'anillos']
<class 'list'>
['ciencia', 'el', 'sol', 'es', 'una', 'estrella', 'que', 'irradia', 'calor', 'y', 'energía', 'por', 'toda', 'la', 'vía', 'lactea', '

In [6]:
stopword_es = nltk.corpus.stopwords.words('spanish')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword_es]
    return text

df['Sin_stopwords'] = df['Words_tokenized'].apply(lambda x: remove_stopwords(x))

In [7]:
df.head()

,Género,Ratings,Documentos,Genero_+_documento,Words_tokenized,Sin_stopwords
0,Salud,7,Los grupos musculares se ejercitan en función ...,salud los grupos musculares se ejercitan en fu...,"[salud, los, grupos, musculares, se, ejercitan...","[salud, grupos, musculares, ejercitan, función..."
1,Salud,5,Existen dos tipos de actividades física que se...,salud existen dos tipos de actividades física ...,"[salud, existen, dos, tipos, de, actividades, ...","[salud, existen, dos, tipos, actividades, físi..."
2,Salud,6,Los deportes más practicados a nivel mundial s...,salud los deportes más practicados a nivel mun...,"[salud, los, deportes, más, practicados, a, ni...","[salud, deportes, practicados, nivel, mundial,..."
3,Cine,5,Las películas con más Oscars de la historia so...,cine las películas con más oscars de la histor...,"[cine, las, películas, con, más, oscars, de, l...","[cine, películas, oscars, historia, titanic, b..."
4,Ciencia,6,El sol es una estrella que irradia calor y ene...,ciencia el sol es una estrella que irradia cal...,"[ciencia, el, sol, es, una, estrella, que, irr...","[ciencia, sol, estrella, irradia, calor, energ..."


In [8]:
def spanish_stemmer(x):
    spanish_stemmer_ = SnowballStemmer(language = 'spanish')
    for i in range(len(x)):
#         print(i)
        for l in range(len(x[i])):
#             print(l)
            x[i][l] = spanish_stemmer_.stem(x[i][l])
#             print(x[i])
    return x
    
df['words_stemmed'] = spanish_stemmer(df['Sin_stopwords'])

In [9]:
df.head()

,Género,Ratings,Documentos,Genero_+_documento,Words_tokenized,Sin_stopwords,words_stemmed
0,Salud,7,Los grupos musculares se ejercitan en función ...,salud los grupos musculares se ejercitan en fu...,"[salud, los, grupos, musculares, se, ejercitan...","[salud, grup, muscular, ejercit, funcion, ejer...","[salud, grup, muscular, ejercit, funcion, ejer..."
1,Salud,5,Existen dos tipos de actividades física que se...,salud existen dos tipos de actividades física ...,"[salud, existen, dos, tipos, de, actividades, ...","[salud, exist, dos, tip, activ, fisic, suel, p...","[salud, exist, dos, tip, activ, fisic, suel, p..."
2,Salud,6,Los deportes más practicados a nivel mundial s...,salud los deportes más practicados a nivel mun...,"[salud, los, deportes, más, practicados, a, ni...","[salud, deport, practic, nivel, mundial, futbo...","[salud, deport, practic, nivel, mundial, futbo..."
3,Cine,5,Las películas con más Oscars de la historia so...,cine las películas con más oscars de la histor...,"[cine, las, películas, con, más, oscars, de, l...","[cin, pelicul, oscars, histori, titanic, benhu...","[cin, pelicul, oscars, histori, titanic, benhu..."
4,Ciencia,6,El sol es una estrella que irradia calor y ene...,ciencia el sol es una estrella que irradia cal...,"[ciencia, el, sol, es, una, estrella, que, irr...","[cienci, sol, estrell, irradi, calor, energ, t...","[cienci, sol, estrell, irradi, calor, energ, t..."


In [10]:


# #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
# tfidf = TfidfVectorizer(analyzer = 'word',lowercase = False, stop_words = stopword_es)

# #Construct the required TF-IDF matrix by fitting and transforming the data
# tfidf_matrix = tfidf.fit_transform(df['Words_tokenized'][0])

# #Output the shape of tfidf_matrix
# tfidf_matrix.shape

In [11]:


#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(analyzer = 'word',lowercase = True, stop_words = stopword_es)

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['words_stemmed'].astype('U'))

#Output the shape of tfidf_matrix
tfidf_matrix.shape

print(tfidf_matrix)

  (0, 68)	0.23955658657259687
  (0, 4)	0.19885253980650688
  (0, 42)	0.23955658657259687
  (0, 33)	0.23955658657259687
  (0, 3)	0.23955658657259687
  (0, 51)	0.19885253980650688
  (0, 49)	0.23955658657259687
  (0, 59)	0.23955658657259687
  (0, 15)	0.23955658657259687
  (0, 50)	0.23955658657259687
  (0, 20)	0.23955658657259687
  (0, 16)	0.39770507961301377
  (0, 27)	0.23955658657259687
  (0, 17)	0.23955658657259687
  (0, 40)	0.23955658657259687
  (0, 30)	0.23955658657259687
  (0, 56)	0.1292685122561784
  (1, 1)	0.3024908939685984
  (1, 47)	0.21462633135822845
  (1, 62)	0.3024908939685984
  (1, 25)	0.2510934196992658
  (1, 0)	0.3024908939685984
  (1, 63)	0.3024908939685984
  (1, 14)	0.3024908939685984
  (1, 24)	0.3024908939685984
  :	:
  (4, 61)	0.5163977794943222
  (4, 9)	0.2581988897471611
  (5, 43)	0.2919126272373849
  (5, 55)	0.2919126272373849
  (5, 66)	0.24231254985824804
  (5, 48)	0.2919126272373849
  (5, 57)	0.2919126272373849
  (5, 38)	0.2919126272373849
  (5, 37)	0.291912627237

In [12]:
# dtm = tf.fit_transform(df[‘Criteria’].values.astype(‘U’))
# dtm = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf.get_feature_names_out()) para la versión del paquete más actual
dtm = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf.get_feature_names())
dtm

,activ,aerob,alt,alta,anaerob,anill,baloncest,benhur,calor,cienci,...,si,sistem,sol,suel,tip,titanic,tod,vari,via,vicevers
0,0.000000,0.000000,0.000000,0.239557,0.198853,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.239557,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.239557
1,0.302491,0.302491,0.000000,0.000000,0.251093,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.302491,0.302491,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.343327,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.353553,0.000000,0.353553,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.353553,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.258199,0.258199,...,0.000000,0.258199,0.516398,0.000000,0.000000,0.000000,0.258199,0.000000,0.258199,0.000000
5,0.000000,0.000000,0.291913,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.242313,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.281924,0.000000,0.000000


In [13]:
# https://goodboychan.github.io/python/datacamp/natural_language_processing/2020/07/17/04-TF-IDF-and-similarity-scores.html

# Similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

[[1.         0.27075317 0.02394891 0.         0.         0.04072492
  0.07975241]
 [0.27075317 1.         0.08252376 0.         0.         0.15672048
  0.02991509]
 [0.02394891 0.08252376 1.         0.         0.         0.17787783
  0.03395364]
 [0.         0.         0.         1.         0.         0.
  0.        ]
 [0.         0.         0.         0.         1.         0.
  0.        ]
 [0.04072492 0.15672048 0.17787783 0.         0.         1.
  0.12605156]
 [0.07975241 0.02991509 0.03395364 0.         0.         0.12605156
  1.        ]]


In [21]:
nn = NearestNeighbors(n_neighbors=1, algorithm='brute', metric='cosine')

nn.fit(dtm.values) # Le pongo values para que no aparezca el warning de los feature names
# https://stackoverflow.com/questions/69326639/sklearn-warning-valid-feature-names-in-version-1-0

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=1)

In [23]:
def recomendar_documentos():
    peticion = input('Indique sobre qué quiere una recomendación: \n')
    
    tokenized = word_tokenize(str.lower(peticion))
    
    no_stop_word = remove_stopwords(tokenized)
    
    stemmed_words = [SnowballStemmer(language = 'spanish').stem(x) for x in no_stop_word]
    
    recomendacion = nn.kneighbors(tfidf.transform(stemmed_words).todense())
    
    print(recomendacion)
    
    return df['Documentos'][recomendacion[1][0]]
#     return df['Documentos'][recomendacion[1][0]],df['Documentos'][recomendacion[1][1]], df['Documentos'][recomendacion[1][2]]

In [24]:
# Las películas del cine más importantes de la historia

recomendar_documentos()

Indique sobre qué quiere una recomendación: 
Las películas del cine más importantes de la historia
(array([[0.64644661],
       [0.64644661],
       [0.6603679 ],
       [0.64644661]]), array([[3],
       [3],
       [6],
       [3]], dtype=int64))


3    Las películas con más Oscars de la historia so...
Name: Documentos, dtype: object

In [ ]:
# Las películas del cine más importantes de la historia